In [38]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from tensorflow.keras import layers, models
from keras.utils import to_categorical

In [39]:
events_dict = {
    'BLUE: first_blood': 0,
    'BLUE: dragon': 1,
    'BLUE: herald': 2,
    'BLUE: first_tower_top': 3,
    'BLUE: first_tower_mid': 4,
    'BLUE: first_tower_bot': 5,
    'BLUE: second_tower_top': 6,
    'BLUE: second_tower_mid': 7,
    'BLUE: second_tower_bot': 8,
    'BLUE: third_tower_top': 9,
    'BLUE: third_tower_mid': 10,
    'BLUE: third_tower_bot': 11,
    'BLUE: inhibitor_top': 12,
    'BLUE: inhibitor_mid': 13,
    'BLUE: inhibitor_bot': 14,
    'BLUE: baron': 15,
    'BLUE: elder_dragon': 16,
    'BLUE: nexus_tower': 17,
    'BLUE: nexus': 18,
    'RED: first_blood': 19,
    'RED: dragon': 20,
    'RED: herald': 21,
    'RED: first_tower_top': 22,
    'RED: first_tower_mid': 23,
    'RED: first_tower_bot': 24,
    'RED: second_tower_top': 25,
    'RED: second_tower_mid': 26,
    'RED: second_tower_bot': 27,
    'RED: third_tower_top': 28,
    'RED: third_tower_mid': 29,
    'RED: third_tower_bot': 30,
    'RED: inhibitor_top': 31,
    'RED: inhibitor_mid': 32,
    'RED: inhibitor_bot': 33,
    'RED: baron': 34,
    'RED: elder_dragon': 35,
    'RED: nexus_tower': 36,
    'RED: nexus': 37
}

In [40]:
df = pd.read_csv('data/clean-one-line.csv')
df = df.drop('game', axis=1)
data = df.values.tolist()

for game in data:
    for i, s in enumerate(game):
            game[i] = s

dataArray = []

for game in data:
    dataArray += game

dataArray = [b for b in dataArray if not(isinstance(b, float))]

In [41]:
df = pd.read_csv('data/clean-one-line.csv')
df = df.drop('game', axis=1)
data = df.values.tolist()

for game in data:
    for i, s in enumerate(game):
        if s in events_dict:
            game[i] = events_dict.get(s)

labelsArray = []

for game in data:
    labelsArray += game

sequence = [b for b in labelsArray if not(isinstance(b, float))]

In [42]:

from random import randint
from numpy import array
from numpy import argmax
from pandas import DataFrame
from pandas import concat
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import RepeatVector
 
# generate a sequence of random numbers in [0, 99]
def generate_sequence():
	finalNum = randint(25, 6996)
	startNum = finalNum - 25
	return sequence[startNum:finalNum]
 
# one hot encode sequence
def one_hot_encode(sequence, n_unique=38):
	encoding = list()
	for value in sequence:
		vector = [0 for _ in range(n_unique)]
		vector[value] = 1
		encoding.append(vector)
	return array(encoding)
 
# decode a one hot encoded string
def one_hot_decode(encoded_seq):
	return [argmax(vector) for vector in encoded_seq]
 
# convert encoded sequence to supervised learning
def to_supervised(sequence, n_in, n_out):
	# create lag copies of the sequence
	df = DataFrame(sequence)
	df = concat([df.shift(n_in-i-1) for i in range(n_in)], axis=1)
	# drop rows with missing values
	df.dropna(inplace=True)
	# specify columns for input and output pairs
	values = df.values
	width = sequence.shape[1]
	X = values.reshape(len(values), n_in, width)
	y = values[:, -:(n_out*width)].reshape(len(values), n_out, width)
	return X, y
 
# prepare data for the LSTM
def get_data(n_in, n_out, sequence = []):
	# generate random sequence
	if not len(sequence):
		sequence = generate_sequence()
	# one hot encode
	encoded = one_hot_encode(sequence)
	# convert to X,y pairs
	X,y = to_supervised(encoded, n_in, n_out)
	return X,y

In [43]:
# define LSTM
n_in = 5
n_out = 2
encoded_length = 38
batch_size = 21
model = Sequential()
model.add(LSTM(150, batch_input_shape=(batch_size, n_in, encoded_length), stateful=True))
model.add(RepeatVector(n_out))
model.add(LSTM(150, return_sequences=True, stateful=True))
model.add(TimeDistributed(Dense(encoded_length, activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# train LSTM
for epoch in range(5000):
	# generate new random sequence
	X,y = get_data(n_in, n_out)
	# fit model for one epoch on this sequence
	model.fit(X, y, epochs=1, batch_size=batch_size, verbose=0, shuffle=False)
	model.reset_states()

# evaluate LSTM
X,y = get_data(n_in, n_out)
yhat = model.predict(X, batch_size=batch_size, verbose=1)
# decode all pairs
for i in range(len(X)):
	print('Expected:', one_hot_decode(y[i]), 'Predicted', one_hot_decode(yhat[i]))

1/1 [==============================] - 1s 1s/step
Expected: [25, 28] Predicted [25, 28]
Expected: [28, 31] Predicted [28, 31]
Expected: [31, 26] Predicted [31, 26]
Expected: [26, 29] Predicted [26, 29]
Expected: [29, 32] Predicted [29, 32]
Expected: [32, 36] Predicted [32, 36]
Expected: [36, 16] Predicted [36, 16]
Expected: [16, 36] Predicted [16, 36]
Expected: [36, 15] Predicted [36, 15]
Expected: [15, 10] Predicted [15, 10]
Expected: [10, 32] Predicted [10, 32]
Expected: [32, 37] Predicted [32, 37]
Expected: [37, 21] Predicted [37, 21]
Expected: [21, 0] Predicted [21, 0]
Expected: [0, 1] Predicted [0, 1]
Expected: [1, 5] Predicted [1, 5]
Expected: [5, 22] Predicted [5, 22]
Expected: [22, 8] Predicted [22, 8]
Expected: [8, 23] Predicted [8, 23]
Expected: [23, 1] Predicted [23, 1]
Expected: [1, 3] Predicted [1, 3]


In [44]:
model.get_config()

{'name': 'sequential_10',
 'layers': [{'class_name': 'InputLayer',
   'config': {'batch_input_shape': (21, 5, 38),
    'dtype': 'float32',
    'sparse': False,
    'ragged': False,
    'name': 'lstm_20_input'}},
  {'class_name': 'LSTM',
   'config': {'name': 'lstm_20',
    'trainable': True,
    'batch_input_shape': (21, 5, 38),
    'dtype': 'float32',
    'return_sequences': False,
    'return_state': False,
    'go_backwards': False,
    'stateful': True,
    'unroll': False,
    'time_major': False,
    'units': 150,
    'activation': 'tanh',
    'recurrent_activation': 'sigmoid',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'GlorotUniform',
     'config': {'seed': None}},
    'recurrent_initializer': {'class_name': 'Orthogonal',
     'config': {'gain': 1.0, 'seed': None}},
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'unit_forget_bias': True,
    'kernel_regularizer': None,
    'recurrent_regularizer': None,
    'bias_regularizer': None,
    